In [7]:
##import torch and geometric libraries and packages

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GATConv


In [13]:
from torch_geometric.utils import to_networkx

In [18]:
##download datasets
dataset_cora = Planetoid(root="./data/Cora", name='Cora')

##interactive data visualization to get insights from "cora" dataset


In [19]:
##transfer cora dataset to network format
cora = to_networkx(dataset_cora.data)

In [20]:
##check if it's directed graph
print(cora.is_directed())


True


In [22]:
##check how mang classes in cora dataset
node_classes = dataset_cora.data.y.data.numpy()
print(node_classes)

[3 4 4 ... 3 3 3]
